In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

In [2]:
def load_data(path):
    with open(path) as fil:
        data = pd.read_csv(fil,sep='\t',header=None,names=["hi","en",""],skip_blank_lines=True,index_col=None)
    data = data[data['hi'].notna()]
    data = data[data['en'].notna()]
    data = data[['hi','en']]
    return data

In [3]:
train = load_data("hi.translit.sampled.train.tsv")
dev = load_data("hi.translit.sampled.dev.tsv")
test = load_data("hi.translit.sampled.test.tsv")

In [4]:
train.head()

,hi,en
0,अं,an
1,अंकगणित,ankganit
2,अंकल,uncle
3,अंकुर,ankur
4,अंकुरण,ankuran


In [5]:
x = train['en'].values
y = train['hi'].values
y = '\t'+y+'\n'

In [6]:
english_tokens = set()
hindi_tokens = set()

for xx,yy in zip(x,y):
    for ch in xx:
        english_tokens.add(ch)
    for ch in yy:
        hindi_tokens.add(ch)
    
english_tokens = sorted(list(english_tokens))
hindi_tokens = sorted(list(hindi_tokens))

In [7]:
eng_token_map = dict([(ch,i+1) for i,ch in enumerate(english_tokens)])
hin_token_map = dict([(ch,i+1) for i,ch in enumerate(hindi_tokens)])

In [8]:
hin_token_map[" "] = 0
eng_token_map[" "] = 0

In [9]:
max_eng_len = max([len(i) for i in x])
max_hin_len = max([len(i) for i in y])

In [10]:
def process(data):
    x,y = data['en'].values, data['hi'].values
    y = "\t" + y + "\n"
    
    a = np.zeros((len(x),max_eng_len),dtype="float32")
    b = np.zeros((len(y),max_hin_len),dtype="float32")
    c = np.zeros((len(y),max_hin_len,len(hindi_tokens)+1),dtype="int")
    
    
    for i,(xx,yy) in enumerate(zip(x,y)):
        for j,ch in enumerate(xx):
            a[i,j] = eng_token_map[ch]

        a[i,j+1:] = eng_token_map[" "]
        for j,ch in enumerate(yy):
            b[i,j] = hin_token_map[ch]

            if j>0:
                c[i,j-1,hin_token_map[ch]] = 1

        b[i,j+1:] = hin_token_map[" "]
        c[i,j:,hin_token_map[" "]] = 1
        
    return a,b,c

In [11]:
trainx, trainxx, trainy = process(train)
valx, valxx, valy = process(dev)
testx,testxx,testy = process(test)

In [12]:
np.random.seed(42)

In [13]:
reverse_eng_map = dict([(i,char) for char,i in eng_token_map.items()])
reverse_hin_map = dict([(i,char) for char,i in hin_token_map.items()])

In [14]:
import keras
import tensorflow as tf
from keras.layers import SimpleRNN,LSTM,GRU,Embedding,Dense,Dropout,Input
from keras.optimizers import Adam,Nadam
from keras import Model

In [15]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [15]:
def build_model(cell = "LSTM",nunits = 32, enc_layers = 1, dec_layers = 1,embed_dim = 32,dense_size=32,dropout=None):
    keras.backend.clear_session()
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(input_dim=len(english_tokens)+1,output_dim = embed_dim,mask_zero=True)
    encoder_context = encoder_embedding(encoder_inputs)
    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(input_dim = len(hindi_tokens)+1,output_dim = embed_dim,mask_zero=True)
    decoder_context = decoder_embedding(decoder_inputs)
    
    if cell == "LSTM":
        encoder_prev = [LSTM(nunits,return_sequences=True) for i in range(enc_layers-1)]
        encoder_fin = LSTM(nunits,return_state=True)
        temp = encoder_context
        for lay in encoder_prev:
            temp = lay(temp)
            if dropout is not None:
                temp = Dropout(dropout)(temp)
            
        _, state_h,state_c = encoder_fin(temp)
        encoder_states = [state_h,state_c]
        
        decoder = [LSTM(nunits,return_sequences=True,return_state=True) for i in range(dec_layers)]
        
        temp,sh,sc = decoder[0](decoder_context,initial_state=encoder_states)
        for i in range(1,dec_layers):
            temp,sh,sc = decoder[i](temp,initial_state=encoder_states)
            
    elif cell == "Simple":
        encoder_prev = [SimpleRNN(nunits,return_sequences=True) for i in range(enc_layers-1)]
        encoder_fin = SimpleRNN(nunits,return_state=True)
        temp = encoder_context
        for lay in encoder_prev:
            temp = lay(temp)
            if dropout is not None:
                temp = Dropout(dropout)(temp)
            
        _, state = encoder_fin(temp)
        encoder_states = state
        
        decoder = [SimpleRNN(nunits,return_sequences=True,return_state=True) for i in range(dec_layers)]
        
        temp,s = decoder[0](decoder_context,initial_state=state)
        for i in range(1,dec_layers):
            temp,s = decoder[i](temp,initial_state=state)
       
    elif cell == "GRU":
        encoder_prev = [GRU(nunits,return_sequences=True) for i in range(enc_layers-1)]
        encoder_fin = GRU(nunits,return_state=True)
        temp = encoder_context
        for lay in encoder_prev:
            temp = lay(temp)
            if dropout is not None:
                temp = Dropout(dropout)(temp)
            
        _, state = encoder_fin(temp)
        encoder_states = state
        
        decoder = [GRU(nunits,return_sequences=True,return_state=True) for i in range(dec_layers)]
        
        temp,s = decoder[0](decoder_context,initial_state=state)
        for i in range(1,dec_layers):
            temp,s = decoder[i](temp,initial_state=state)
            
        
    dense_lay1 = Dense(dense_size,activation='relu')
    pre_out = dense_lay1(temp)
    dense_lay2 = Dense(len(hindi_tokens)+1,activation = 'softmax')
    final_output = dense_lay2(pre_out)
    
    train = Model([encoder_inputs,decoder_inputs],final_output)
    
    encoder_model = Model(encoder_inputs,encoder_states)
    
    if cell == "LSTM":
        state_inputs = []
        state_outputs = []
        
        decoder_input_h = Input(shape=(nunits,))
        decoder_input_c = Input(shape=(nunits,))
        temp,sh,sc = decoder[0](decoder_context,initial_state = [decoder_input_h,decoder_input_c])
        state_inputs += [decoder_input_h,decoder_input_c]
        state_outputs += [sh,sc]
                                                                  
        for i in range(1,dec_layers):
            decoder_input_h = Input(shape=(nunits,))
            decoder_input_c = Input(shape=(nunits,))
            temp,sh,sc = decoder[i](temp,initial_state = [decoder_input_h,decoder_input_c])
            state_inputs += [decoder_input_h,decoder_input_c]
            state_outputs += [sh,sc]
            
        decoder_input_pass = [decoder_inputs] + state_inputs
        
    elif cell == "Simple":
        state_inputs = []
        state_outputs = []
        
        state_input = Input(shape=(nunits,))
        temp,s = decoder[0](decoder_context,initial_state = state_input)
        state_inputs.append(state_input)
        state_outputs.append(s)
                                                                  
        for i in range(1,dec_layers):
            state_input = Input(shape=(nunits,))
            temp,s = decoder[i](temp,initial_state = state_input)
            state_inputs.append(state_input)
            state_outputs.append(s)
            
        decoder_input_pass = [decoder_inputs] + state_inputs
        
    elif cell == "GRU":
        state_inputs = []
        state_outputs = []
        
        state_input = Input(shape=(nunits,))
        temp,s = decoder[0](decoder_context,initial_state = state_input)
        state_inputs.append(state_input)
        state_outputs.append(s)
                                                                  
        for i in range(1,dec_layers):
            state_input = Input(shape=(nunits,))
            temp,s = decoder[i](temp,initial_state = state_input)
            state_inputs.append(state_input)
            state_outputs.append(s)
            
        decoder_input_pass = [decoder_inputs] + state_inputs

    pre_out = dense_lay1(temp)
    final_output = dense_lay2(pre_out)
    
    decoder_model = Model(decoder_input_pass, [final_output]+state_outputs)
    
    return train,encoder_model,decoder_model

In [16]:
import wandb
from wandb.keras import WandbCallback

In [17]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'lr': {
            'values': [1e-2,1e-3]
        },
        'dense_size': {
            'values': [64,128,512,1024]
        },
        'dropout': {
            'values': [0.0,0.2,0.5]
        },
        'nunits': {
            'values': [64,128,256,512]
        },
        'cell': {
            'values': ["LSTM","GRU","Simple"]
        },
        'embed_size': {
            'values': [32,64,128,256]
        },
        'enc_layers': {
            'values': [1,3,5]
        },
        'dec_layers': {
            'values': [1,3,5]
        },
        
    }
}


In [21]:
sweep_id = wandb.sweep(sweep_config,entity = "kris21" , project = "cs6910-a3")

Create sweep with ID: 1veefb4i
Sweep URL: https://wandb.ai/kris21/cs6910-a3/sweeps/1veefb4i


In [19]:
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'lr': 1e-2,
        'dense_size': 128,
        'nunits': 128,
        'cell': 'LSTM',
        'embed_size': 64,
        'enc_layers': 1,
        'dec_layers': 1,
        'dropout': 0.
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults,name="cs6910-a2")
    
    cfg = wandb.config
    
    name = f'nunits_{cfg.nunits}_cell_{cfg.cell}_encl_{cfg.enc_layers}_decl_{cfg.dec_layers}_emb_{cfg.embed_size}_ds_{cfg.dense_size}_do_{cfg.dropout}_lr_{cfg.lr}'
    wandb.run.name = name
    wandb.run.save()
    
    # Config is a variable that holds and saves hyperparameters and inputs
    
    train,enc,dec = build_model(nunits=cfg.nunits,
                                dense_size=cfg.dense_size,
                                enc_layers=cfg.enc_layers,
                                dec_layers=cfg.dec_layers,
                                cell = cfg.cell,
                                dropout = cfg.dropout,
                                embed_dim = cfg.embed_size)
    train.compile(optimizer = Adam(lr=cfg.lr),loss='categorical_crossentropy',metrics=['accuracy'])
    print("model building done")
    train.fit([trainx,trainxx],trainy,
             batch_size=64,
             validation_data = ([valx,valxx],valy),
             epochs=20,
             callbacks = [WandbCallback(monitor='val_accuracy',mode='min')])
    print("model training done")


In [22]:
wandb.agent(sweep_id, train,count=32)

wandb: Agent Starting Run: hy67ky6s with config:
wandb: 	cell: GRU
wandb: 	dec_layers: 3
wandb: 	dense_size: 1024
wandb: 	dropout: 0.2
wandb: 	embed_size: 128
wandb: 	enc_layers: 5
wandb: 	lr: 0.01
wandb: 	nunits: 64
wandb: Currently logged in as: kris21 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.28 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


model building done
Epoch 1/20
691/691 [==============================] - 77s 63ms/step - loss: 0.8908 - accuracy: 0.3924 - val_loss: 0.4346 - val_accuracy: 0.6543
Epoch 2/20
691/691 [==============================] - 34s 49ms/step - loss: 0.4150 - accuracy: 0.6736 - val_loss: 0.3186 - val_accuracy: 0.7374
Epoch 3/20
691/691 [==============================] - 33s 48ms/step - loss: 0.3213 - accuracy: 0.7394 - val_loss: 0.2824 - val_accuracy: 0.7653
Epoch 4/20
691/691 [==============================] - 33s 48ms/step - loss: 0.2799 - accuracy: 0.7702 - val_loss: 0.2637 - val_accuracy: 0.7801
Epoch 5/20
691/691 [==============================] - 33s 48ms/step - loss: 0.2580 - accuracy: 0.7870 - val_loss: 0.2682 - val_accuracy: 0.7842
Epoch 6/20
691/691 [==============================] - 34s 48ms/step - loss: 0.2490 - accuracy: 0.7962 - val_loss: 0.2413 - val_accuracy: 0.7966
Epoch 7/20
691/691 [==============================] - 33s 48ms/step - loss: 0.2359 - accuracy: 0.8048 - val_loss: 0.

epoch,19
loss,0.20177
accuracy,0.83335
val_loss,0.22723
val_accuracy,0.81467
_runtime,726
_timestamp,1619936433
_step,19
best_val_accuracy,0.65429
best_epoch,0


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▅▆▇▇▇▇▇████████████
val_loss,█▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▆▆▇▇▇▇▇███████████
_runtime,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_timestamp,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██


...

wandb: Agent Starting Run: usf0qv9h with config:
wandb: 	cell: LSTM
wandb: 	dec_layers: 3
wandb: 	dense_size: 1024
wandb: 	dropout: 0.2
wandb: 	embed_size: 128
wandb: 	enc_layers: 3
wandb: 	lr: 0.001
wandb: 	nunits: 64
wandb: wandb version 0.10.28 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


model building done
Epoch 1/20
691/691 [==============================] - 60s 51ms/step - loss: 1.1196 - accuracy: 0.2717 - val_loss: 0.7263 - val_accuracy: 0.4484
Epoch 2/20
691/691 [==============================] - 28s 40ms/step - loss: 0.7118 - accuracy: 0.4743 - val_loss: 0.5268 - val_accuracy: 0.5863
Epoch 3/20
691/691 [==============================] - 28s 40ms/step - loss: 0.5357 - accuracy: 0.5900 - val_loss: 0.4151 - val_accuracy: 0.6623
Epoch 4/20
691/691 [==============================] - 28s 40ms/step - loss: 0.4205 - accuracy: 0.6696 - val_loss: 0.3272 - val_accuracy: 0.7324
Epoch 5/20
691/691 [==============================] - 28s 40ms/step - loss: 0.3411 - accuracy: 0.7297 - val_loss: 0.2774 - val_accuracy: 0.7750
Epoch 6/20
691/691 [==============================] - 28s 40ms/step - loss: 0.2862 - accuracy: 0.7701 - val_loss: 0.2439 - val_accuracy: 0.8018
Epoch 7/20
691/691 [==============================] - 28s 40ms/step - loss: 0.2553 - accuracy: 0.7940 - val_loss: 0.

epoch,19
loss,0.11326
accuracy,0.90465
val_loss,0.1665
val_accuracy,0.86434
_runtime,598
_timestamp,1619977957
_step,19
best_val_accuracy,0.44843
best_epoch,0


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▆▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
accuracy,▁▃▄▅▆▆▇▇▇▇▇▇████████
val_loss,█▆▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▅▆▆▇▇▇▇███████████
_runtime,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_timestamp,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██


In [52]:
train,enc,dec = build_model(nunits=64,dense_size=128,enc_layers=2,dec_layers=2,cell = "GRU")

In [53]:
train.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [54]:
train.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 32)     864         input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
gru (GRU)                       (None, None, 64)     18816       embedding[0][0]                  
______________________________________________________________________________________________

In [66]:
train.fit([trainx,trainxx],trainy,
         batch_size=32,
         validation_data=([valx,valxx],valy),
         epochs=20)

Epoch 1/20
1382/1382 [==============================] - 32s 23ms/step - loss: 0.3894 - accuracy: 0.7026 - val_loss: 0.3317 - val_accuracy: 0.7379
Epoch 2/20
1382/1382 [==============================] - 32s 23ms/step - loss: 0.3111 - accuracy: 0.7613 - val_loss: 0.2789 - val_accuracy: 0.7790
Epoch 3/20
1382/1382 [==============================] - 32s 23ms/step - loss: 0.2674 - accuracy: 0.7931 - val_loss: 0.2445 - val_accuracy: 0.8043
Epoch 4/20
1382/1382 [==============================] - 32s 23ms/step - loss: 0.2362 - accuracy: 0.8151 - val_loss: 0.2300 - val_accuracy: 0.8143
Epoch 5/20
1382/1382 [==============================] - 32s 23ms/step - loss: 0.2140 - accuracy: 0.8307 - val_loss: 0.2157 - val_accuracy: 0.8230
Epoch 6/20
1382/1382 [==============================] - 32s 23ms/step - loss: 0.1978 - accuracy: 0.8421 - val_loss: 0.1984 - val_accuracy: 0.8374
Epoch 7/20
1382/1382 [==============================] - 32s 23ms/step - loss: 0.1851 - accuracy: 0.8513 - val_loss: 0.1973 -

## Inference

In [74]:
def beam_search(inp_seq,k,dec_layers,cell="LSTM"):
    statess = enc.predict(inp_seq)
    target_seq = np.zeros((1,1))
    target_seq[0,0] = hin_token_map["\t"]
    if cell == "LSTM":
        states = []
        for i in range(dec_layers):
            states += [statess[0],statess[1]]
    else:
        states = []
        for i in range(dec_layers):
            states += [statess]
            
    output = dec.predict([target_seq]+states)
    states = output[1:]
    
    best_chars = np.argsort(output[0][0,-1,:])[::-1][:k]
    scores = np.sort(output[0][0,-1,:])[::-1][:k]
    sequences = [([ch],-np.log(sc),states) for ch,sc in zip(best_chars,scores)]
    
    
    for t1 in range(max_hin_len-1):
        candidates = []
        for seq,sc,st in sequences:
            if seq[-1] == hin_token_map["\n"]:
                candidates.append((seq,sc,st))
                continue
            target_seq[0,0] = seq[-1]
            output = dec.predict([target_seq]+st)
            best_chars = np.argsort(output[0][0,-1,:])[::-1][:k]
            scores = np.sort(output[0][0,-1,:])[::-1][:k]
            
            for ch,scor in zip(best_chars,scores):
                candidates.append((seq+[ch],sc-np.log(scor),output[1:]))
                
        candidates = sorted(candidates,key = lambda tup:tup[1])
        if len(candidates) > 0:
            sequences = candidates[:k]
        
    seq,sc,_ = sequences[0]
#     ret = ""
#     for ch in seq:
#         if ch == hin_token_map["\n"]:
#             break
#         ret += reverse_hin_map[ch]
        
    return seq,sc

In [67]:
def inference(inp,dec_layers,cell="LSTM"):
    statess = enc.predict(inp)
    target_seq = np.zeros((inp.shape[0],1))
    target_seq[:,0] = hin_token_map["\t"]
    
    states = []
    
    if cell == "LSTM":
        for c in range(dec_layers):
            states += [statess[0],statess[1]]
            
    else:
        for c in range(dec_layers):
            states += [statess]
            
    ans = np.zeros((inp.shape[0],max_hin_len))
    
    for i in range(max_hin_len):
        output = dec.predict([target_seq]+states)
        ans[:,i] = np.argmax(output[0][:,-1,:],axis=1)
        target_seq[:,0] = ans[:,i]
        states = output[1:]
        
    return ans   

In [68]:
pred = inference(testx,2,cell="GRU")

In [70]:
import time

In [69]:
for i in range(10):
    idx = np.random.choice(testx.shape[0])
    orig = ""
    for ch in testx[idx]:
        orig += reverse_eng_map[ch]
        if reverse_eng_map[ch] == "\n":
            break
        
    deco = ""
    for ch in pred[idx]:
        deco += reverse_hin_map[ch]
        if reverse_hin_map[ch] == "\n":
            break
        
    print("Original sentence:",orig)
    print("Decoded sentence:", deco)
    print("=========")

Original sentence: end                 
Decoded sentence: ईंड

Original sentence: menon               
Decoded sentence: मेनों

Original sentence: chithrkar           
Decoded sentence: चित्रकर

Original sentence: sooraj              
Decoded sentence: सूरज

Original sentence: rajneetikaran       
Decoded sentence: राजनीतिवासार

Original sentence: prekshakon          
Decoded sentence: प्रेक्षाकों

Original sentence: dalbadl             
Decoded sentence: डालबडल

Original sentence: kambalon            
Decoded sentence: कांबलों

Original sentence: bahuly              
Decoded sentence: बहुल्य

Original sentence: gtod                
Decoded sentence: ग्रोड



In [73]:
tot = 0
for i in range(5):
    idx = np.random.choice(testx.shape[0])
    start = time.time()
    pred = beam_search(testx[idx:idx+1],1,2,cell="GRU")[0]
    end = time.time()
    tot += end-start
    orig = ""
    for ch in testx[idx]:
        orig += reverse_eng_map[ch]
        
    print("Original sentence:",orig)
    print("Decoded sentence:", pred)
    print("=========")
    
print(tot/5)

Original sentence: uplakshya           
Decoded sentence: उपलक्ष्य
Original sentence: bhugata             
Decoded sentence: भुगता
Original sentence: dhokebaaj           
Decoded sentence: धोकेबाज
Original sentence: putul               
Decoded sentence: पुटूब
Original sentence: brahmalin           
Decoded sentence: ब्रह्मालीन
0.39509596824646


In [75]:
pred = [beam_search(testx[id:id+1],3,2,cell="GRU")[0] for id in range(testx.shape[0])]

In [77]:
acc = 0

for i,pr in enumerate(pred):
    fl = 1
    for j,ch in enumerate(pr):
        if ch != np.argmax(testy[i,j,:]):
            fl = 0
            break
            
    if fl==1:
        acc+=1
        
        
print(acc/len(pred))

0.3494002665482008
